This function computes closed null geodesics of $ C_{t_0}^{t_N}(\mathbf{x})-\lambda^2 \mathbf{I} $, where $ C_{t_0}^{t_N}(\mathbf{x}, t) = \begin{pmatrix} C^{11} && C^{12} \\ C^{12} && C^{22} \end{pmatrix} $ is the (symmetric) Cauchy Green strain tensor.

| Name | Type (Shape) | Description |
| --- | --- | --- |
| X | array (Ny, Nx) | X-meshgrid|
| Y | array (Ny, Nx) | Y-meshgrid|
| lam | float | $ \lambda = $ stretching|
| interp_phi_prime | interpolant | $ \dot{\phi} $|
| d_threshold | float | threshold distance between start point and return point of the closed curve |
| C11 | array (Ny,Nx) | array containing the C11-element of the Cauchy Green strain tensor |
| iteration_length | float | length of dummy time interval used for computation of elliptic LCSs |
| n_iterations | int | resolution of dummy time interval |
| verbose | bool | if verbose=True --> function reports progress at every 100th iteration. |
| solODE_closed_curve | list | list containing the closed solution curves ($ \mathbf{x}, \phi $) |
| x0lam | list | list containing x-coordinates of initial conditions $ x_0(\lambda) $ |
| y0lam | list | list containing y-coordinates of initial conditions $ y_0(\lambda) $ |

In [1]:
import sys, os

# get current directory
path = os.getcwd()

# get parent directory
parent_directory = os.path.sep.join(path.split(os.path.sep)[:-3])

# add utils folder to current working path
sys.path.append(parent_directory+"/subfunctions/utils")

In [2]:
# Import numpy
import numpy as np

# function computing initial conditions (depending on \lambda)
from ipynb.fs.defs.init_level_set import _init_level_set

# find closed curve
from ipynb.fs.defs.closed_curve import closed_curve

# RK4 integrator
from ipynb.fs.defs.RK4_integration import RK4_integration

In [3]:
def closed_null_geodesics(X, Y, lam, interp_phi_prime, d_threshold, C11, iteration_length = 12, n_iterations = 4000, verbose = False):
    '''
    Compute closed null geodesics. The algorithm returns closed null geodesics coinciding with elliptic LCS.
    
    Parameters:
        X:                      array (Ny, Nx),  X-meshgrid.
        Y:                      array (Ny, Nx),  Y-meshgrid.
        lam:                    float, lambda --> stretching used to compute ICs of null geodesics.
        interp_phi_prime:       Inteprolant for phi_prime.
        d_threshold:            float, threshold value for re-intersection of closed curve
        C11:                    array (Ny, Nx), array of C_{11}.
        iteration_length:       float, length of dummy time interval used for computation of elliptic LCSs
        n_iterations:           int, resolution of dummy time interval
        verbose:                bool, if True, function reports progress at every 100th iteration.          
        
    Returns:
        solODEcurves:           list, contains solutions of ODE = set of closed null geodesics.
        x0lam:                  list, contains x-coordinates of ICs.
        y0lam:                  list, contains y-coordinates of ICs.
    '''
    
    # domain where the rate of strain field is defined
    defined_domain = np.isfinite(C11).astype(int)
    
    # compute initial conditions
    x0lam, y0lam, phi0lam = _init_level_set(X, Y, C11, lam)
    
    # define integration domain of dummy variable.
    # Note if you do not find any structures, then I recommend increasing this value. 
    # For the majority of examples the algorithm converges much sooner. 
    # An upper bound on the dummy time-interval is included in order to avoid long computation times.
    s = [0, iteration_length]
    
    # define resolution of trajectories
    # This resolution can also be refined.
    s_eval = np.linspace(s[0], s[-1], n_iterations)
    
    ds = s_eval[1]-s_eval[0]
    
    # number of initial conditions
    len_x0lam = len(x0lam)
    
    solODE = np.zeros((len(s_eval), 3, len_x0lam))
    
    # initial conditions
    solODE[0, :, :] = np.array([x0lam, y0lam, phi0lam])
    
    for i in range(len(s_eval)-1):
        
        solODE[i+1, :, :] = RK4_integration(solODE[i, :, :], ds, interp_phi_prime)
    
    solODE_closed_curves = []
    
    print("Computation of closed null geodesics: Started...")
    
    # iterate over all initial conditions [x0lam, y0lam, phi0lam]
    for j in range(len_x0lam):
        
        # store x, y, phi
        # we do not solve the ODE for all ICs as this would require long computation times.
        # for higher accuracy just replace the subsequente three lines of codes with the commented version
        x = solODE[::2,0, j] #solODE[:,0,j]
        y = solODE[::2,1, j] #solODE[:,1,j]
        phi = solODE[::2,2, j] #solODE[:,2,j]
    
        if verbose:
            if j%100==0:
                print("Percentage completed: ", np.around(np.around(j/len_x0lam, 4)*100,2))
    
        # Check if curve is closed after completing one full cycle and find curve with minimum re-intersection distance.
        x_closed, y_closed, phi_closed = closed_curve(x, y, phi, d_threshold, X, Y, defined_domain)
        
        # store solutions
        solODE_closed_curves.append([x_closed, y_closed, phi_closed])
    
    print("Computation of closed null geodesics: Done!")
    
    return solODE_closed_curves, [x0lam, y0lam]